In [7]:
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import create_react_agent
from agent.llm import llm

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI


# 导入文件处理相关库
import os
from pathlib import Path

from langchain_core.messages import SystemMessage, HumanMessage
import os
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader
# 使用PyMuPDFLoader加载PDF文件
import requests
import tempfile


# 合同文件的URL路径
contract_file_path = "https://wangwenjie24.oss-cn-beijing.aliyuncs.com/%E5%9F%BA%E4%BA%8E%E6%95%B0%E5%AD%97%E5%AD%AA%E7%94%9F%E7%9A%84%E7%AB%8B%E4%BD%93%E4%BB%93%E5%BA%93%E5%92%8C%E5%B7%A5%E5%8E%82%E8%BD%A6%E9%97%B4%E5%B1%95%E7%A4%BA%E9%A1%B9%E7%9B%AE-%E6%8A%80%E6%9C%AF%E5%8D%8F%E8%AE%AE%E4%B9%A6-%E5%8C%97%E4%BA%AC%E7%A7%91%E5%AE%89-20241108-V1.8.docx?Expires=1745417898&OSSAccessKeyId=TMP.3KrhPUBNphVcaKFdFBf2EX5gs2jXnRD3AQ4pdNLyWXrqQuf9HYzvvN5sPdE6QaYqSbY2epoNHUV1AFJgdRJV9WEnbZMyvM&Signature=UZ4%2Bj8W%2F03tRDbNcLhMSEf8kuTw%3D"

try:
    # 下载网络URL的文件到临时文件
    response = requests.get(contract_file_path)
    if response.status_code == 200:
        with tempfile.NamedTemporaryFile(delete=False) as temp_file:
            temp_file.write(response.content)
            temp_file_path = temp_file.name
        
        # # 根据文件类型选择合适的加载器
        # # 从URL中获取文件名
        # file_name = contract_file_path.split('/')[-1].split('?')[0]
        # # 从文件名中获取扩展名
        # file_extension = file_name.lower().split('.')[-1] if '.' in file_name else ''
        # 也可以从响应头的Content-Type判断
        content_type = response.headers.get('Content-Type', '')
        if 'pdf' in content_type.lower():
            file_extension = 'pdf'
        elif 'word' in content_type.lower() or 'docx' in content_type.lower():
            file_extension = 'docx'
        
        if file_extension == 'pdf':
            # 使用PyMuPDFLoader加载PDF文件
            from langchain_community.document_loaders import PyMuPDFLoader
            loader = PyMuPDFLoader(temp_file_path)
            documents = loader.load()
            print(f"成功加载PDF合同，共{len(documents)}页")
        elif file_extension in ['doc', 'docx']:
            # 使用UnstructuredWordDocumentLoader加载Word文件
            from langchain_community.document_loaders import UnstructuredWordDocumentLoader
            loader = UnstructuredWordDocumentLoader(temp_file_path)
            documents = loader.load()
            print(f"成功加载Word合同文档")
        else:
            raise ValueError(f"不支持的文件类型: {file_extension}")
            
        # 提取文本内容
        contract_content = "\n\n".join([doc.page_content for doc in documents])
        
        # 删除临时文件
        os.unlink(temp_file_path)
    else:
        contract_content = "无法下载合同文件"
        print(f"无法下载合同文件! 状态码: {response.status_code}")
except Exception as e:
    contract_content = f"处理合同文件时出错: {str(e)}"
    print(f"处理合同文件时出错: {str(e)}")


response = llm.invoke([
        SystemMessage(content="""You are a corporate legal expert with access to check contract risk.
Analyze the legal risks in the contract text from perspectives including payment and settlement terms, delivery and acceptance terms, breach of contract liability, intellectual property clauses, confidentiality provisions, applicable law and dispute resolution clauses, and other terms.
        """),
        HumanMessage(content=f"请站在甲方角度分析以下合同文本中的法律风险:\n\n{contract_content}")
    ])

response

成功加载Word合同文档


AIMessage(content='### 法律风险分析\n\n#### 1. 支付和结算条款\n**风险点：**\n- **付款条件不明确**：虽然合同中列出了详细的付款进度和条件，但某些条件的表述较为模糊，例如“项目上线并整线验收合格后”，可能在实际操作中存在争议。\n- **发票要求**：合同要求乙方提供增值税专用发票，但未明确发票的开具时间和格式要求，可能导致付款延误。\n- **违约金过高**：合同中规定的违约金（如3‰的日违约金）可能被认为过高，法院可能会调整违约金的数额。\n\n**建议：**\n- 明确付款条件的具体标准和流程，避免歧义。\n- 规定发票的开具时间和格式要求，确保付款顺利进行。\n- 考虑调整违约金的比例，使其更加合理，以减少法律风险。\n\n#### 2. 交付和验收条款\n**风险点：**\n- **交付时间不明确**：合同中虽然列出了详细的阶段时间和任务，但未明确每个阶段的具体交付物和验收标准，可能导致交付延误。\n- **验收标准不清晰**：验收标准虽然列在附件中，但未明确具体的验收流程和标准，可能导致验收争议。\n- **验收时间过长**：合同规定甲方应在7个工作日内做出验收决定，但未明确如果甲方未按时验收的后果，可能导致项目延期。\n\n**建议：**\n- 详细列出每个阶段的具体交付物和验收标准，确保双方对交付内容有明确的认识。\n- 规定明确的验收流程和标准，避免验收争议。\n- 设置超时验收的后果，例如自动视为验收合格，以防止项目延期。\n\n#### 3. 违约责任\n**风险点：**\n- **违约金过高**：合同中规定的违约金（如20%的合同总金额）可能被认为过高，法院可能会调整违约金的数额。\n- **违约责任不全面**：合同中虽然列出了多种违约情形，但未涵盖所有可能的违约情况，可能导致某些违约行为无法得到有效制约。\n- **解除合同的条件不明确**：合同中规定了多种解除合同的情形，但未明确解除合同的程序和后果，可能导致解除合同的困难。\n\n**建议：**\n- 调整违约金的比例，使其更加合理，以减少法律风险。\n- 补充其他可能的违约情形，确保合同的全面性。\n- 明确解除合同的程序和后果，确保解除合同的顺利进行。\n\n#### 4. 知识产权条款\n**风险点：**\n- **知识产权归属不明确*

In [12]:
print(response.content)

### 法律风险分析

#### 一、支付和结算条款
1. **付款进度安排**：
   - **风险点**：付款进度较为前置，特别是项目启动后的30%预付款，可能增加甲方的资金压力。如果乙方未能按时完成项目，甲方将面临较大的财务风险。
   - **建议**：考虑调整付款进度，例如将30%的预付款降低至20%，并在项目的关键里程碑节点后再支付剩余的10%。

2. **违约金**：
   - **风险点**：如果乙方未能按时完成项目，每延迟一天按合同总金额的3‰计算违约金，但最高不超过合同总金额的20%。这个违约金上限可能不足以覆盖甲方的实际损失。
   - **建议**：提高违约金的上限，或者增加其他补偿措施，如延期利息等。

#### 二、交付和验收条款
1. **交付时间**：
   - **风险点**：合同规定19周内完成项目，但没有明确具体的每个阶段的时间节点和相应的责任。如果乙方拖延某一个阶段，可能会影响整个项目的进度。
   - **建议**：细化每个阶段的具体时间节点和责任，明确每个阶段的交付物和验收标准，确保项目按计划推进。

2. **验收标准**：
   - **风险点**：验收标准虽然详细，但缺乏具体的量化指标，可能导致验收时的主观判断。
   - **建议**：增加具体的量化指标，如系统响应时间、故障率等，确保验收标准客观、可衡量。

#### 三、违约责任
1. **一般违约责任**：
   - **风险点**：违约责任的描述较为笼统，缺乏具体的违约情形和对应的处罚措施。
   - **建议**：细化具体的违约情形，如未按时交付、交付物不符合要求等，并明确相应的处罚措施。

2. **知识产权侵权**：
   - **风险点**：如果乙方提供的软件侵犯了第三方的知识产权，甲方可能面临法律诉讼和经济损失。
   - **建议**：增加乙方对知识产权侵权的担保条款，明确乙方应承担所有相关的法律责任和费用。

#### 四、知识产权条款
1. **知识产权归属**：
   - **风险点**：合同中明确了乙方已有的框架类软件知识产权归乙方所有，但没有明确乙方是否有权使用这些框架类软件。
   - **建议**：明确乙方有权使用这些框架类软件，并确保这些框架类软件不侵犯第三方的知识产权。

2. **定制开发软件的知识产权**：
   - **

In [4]:
import os
import dashscope
import pyaudio
import time
import base64
import numpy as np


p = pyaudio.PyAudio()
# 创建音频流
stream = p.open(format=pyaudio.paInt16,
                channels=1,
                rate=24000,
                output=True)

text = """
### 法律风险分析报告

#### 一、支付和结算条款
1. **付款条件**：
   - 合同规定了三个付款节点，分别为合同签订后30%、项目上线并验收合格后60%、项目上线使用并验收合格一年后10%。这种分期付款方式有助于降低甲方的风险，但也需要注意以下几点：
     - **付款条件的明确性**：合同中应明确规定每个付款节点的具体条件，避免因条件模糊导致付款争议。
     - **发票要求**：每次付款前，乙方需提供合格的增值税专用发票。甲方应确保发票的真实性和合法性，防止因发票问题导致税务风险。
"""
responses = dashscope.audio.qwen_tts.SpeechSynthesizer.call(
    model="qwen-tts",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    text=text,
    voice="Ethan",
    stream=True
)


In [5]:
for chunk in responses:
    audio_string = chunk["output"]["audio"]["data"]

    wav_bytes = base64.b64decode(audio_string)
    audio_np = np.frombuffer(wav_bytes, dtype=np.int16)
    # 直接播放音频数据
    stream.write(audio_np.tobytes())

time.sleep(0.8)
# 清理资源
stream.stop_stream()
stream.close()
p.terminate()
